In [ ]:
import os
from datetime import date, datetime
import yaml
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from experiment.toolkits.plotting_helper import *

project_dir = Path().resolve().parents[1]
sys.path.insert(0, str(project_dir))
home = os.path.expanduser("~") 
os.chdir(str(project_dir))
import pickle
import datetime
import matplotlib.ticker as ticker
from scipy.stats import linregress
import experiment.toolkits.figure_formatting.formatter_2023_az as fmt
import experiment.toolkits.figure_formatting.production_quality_plots as prod_plot
from experiment.toolkits.plotting_helper import *
from tools.benchmarks.helpers import *

In [ ]:
base_cmd = str(project_dir) + "/bin/modules/operational-benchmarking/op-bench-operations"
def get_operations (algorithm, Ntx, Nty, target, nt, nr, batching, relative = 0, rel_algo = ""):
    command = base_cmd + " " + algorithm + " " + str(Ntx) + " " + str(Nty) + " " + str(target) + " " + str(nt) + " " + str(nr) + " " + str(batching) + " " + str(relative) + " " + rel_algo
    print(command)
    return os.popen(command).read()

In [ ]:
output_2d_batching = get_operations("REDREC-CPU-V3-2D", 32, 54, 1024, 1000, 1, 1, 1)
data_2d_batching = [int(x) for x in filter((lambda x: x != ' '), output_2d_batching.split("\n")[2].split(","))]
output_2d_non_batching = get_operations("REDREC-CPU-V3-2D", 32, 54, 1024, 1000, 1, 0, 1)
data_2d_non_batching = [int(x) for x in filter((lambda x: x != ' '), output_2d_non_batching.split("\n")[2].split(","))]


In [ ]:
data_2d_non_batching = {}
data_2d_batching = []
with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", "edi_non_batched.pickle"), "wb") as file:
    pickle.dump(data_2d_non_batching, file)
with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", "edi_batched.pickle"), "wb") as file:
    pickle.dump(data_2d_batching, file)

In [ ]:

# Generate random data
data = np.random.randn(1000)

# Compute the histogram data
hist, bin_edges = np.histogram(data_2d_batching, bins=7, range= np.mean(data_2d_batching) + 3 * np.std(data_2d_non_batching) * np.array([-1, 1]))
hist_non, bin_edges_non = np.histogram(data_2d_non_batching, bins=7, range= np.mean(data_2d_non_batching) + 3 * np.std(data_2d_non_batching) * np.array([-1, 1]))
print(np.mean(data_2d_batching), np.mean(data_2d_non_batching))
print(np.std(data_2d_batching), np.std(data_2d_non_batching))

# Create a new fig and ax object to be a production quality version
fig, ax = plt.subplots()
# Set the font type and size
plt.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
fontsize = 7
plt.rc('pdf', fonttype=42)  # make the test exportable as text, not an image

# Calculate figure size in inches and set the size of our new figure
figure_width_cm = 4.3
figure_height_cm = 2.8
figure_width_in = figure_width_cm / 2.54
figure_height_in = figure_height_cm / 2.54

fig.set_size_inches((figure_width_in, figure_height_in))

# Set the margins of our production quality figure
left_margin = (4.3-3.48-0.0235)/4.3
right_margin = (4.3-0.05)/4.3
top_margin = (2.75-0.05)/2.8
bottom_margin = (2.75-2.21-0.0235)/2.8

ax.set_position([left_margin, bottom_margin, right_margin - left_margin, top_margin - bottom_margin])

ax.bar(bin_edges[:-1], hist/1000, width=np.diff(bin_edges),  align='center', facecolor='#FFD3AD', edgecolor='#E06900', linewidth=0.2)
ax.bar(bin_edges_non[:-1], hist_non/1000, width=np.diff(bin_edges_non),  align='center', facecolor='#FFF1AD', edgecolor='#E0BB00', linewidth=0.2)
ax.set_xlabel('EDI cycles')
ax.set_ylabel('Frequency')
# Set a blank title for the new figure
ax.set_title('')

if True:
    ax.legend()
    legend = ax.legend(bbox_to_anchor=(0.95, 0.95), loc = 'upper right', borderaxespad=0., fontsize=4, handlelength=0.3, handleheight=0.3)

# Format the ticks as desired for the new figure
ax.tick_params(axis='both', direction='in', length = 2, labelsize = fontsize, pad = 2, width = 0.75)
ax.xaxis.set_ticks_position('both')
ax.yaxis.set_ticks_position('both')

xlim = ax.get_xlim()
ylim = ax.get_ylim()

ticks_x = np.linspace(xlim[0], xlim[1], 5)
ticks_y = np.linspace(ylim[0], ylim[1], 5)
ax.set_xticks(ticks_x)
ax.set_yticks(ticks_y)

ticklabels_x = ticks_x
ticklabels_y = ticks_y
ax.set_xticklabels(ticklabels_x)
ax.set_yticklabels(ticklabels_y)

tick_pad_x = 1.5
tick_pad_y = 1
ax.tick_params('x', labelsize=fontsize, pad=tick_pad_x) 
ax.tick_params('y', labelsize=fontsize, pad=tick_pad_y) 


# Formats the axes lables
label_x = ax.get_xlabel()
label_y = ax.get_ylabel()
label_pad_x = 1
label_pad_y = 1
ax.set_xlabel(label_x, fontsize=fontsize, labelpad=label_pad_x)
ax.set_ylabel(label_y, fontsize=fontsize, labelpad=label_pad_y)

# Sets the xlim and ylim axes so the the ticks are not right to the end (creating the corner square))
epsilon_x = 0.05
epsilon_y = epsilon_x
ax.set_xlim(xlim[0] - epsilon_x * (xlim[1] - xlim[0]), xlim[1] + epsilon_x * (xlim[1] - xlim[0]))
ax.set_ylim(ylim[0] - epsilon_y * (ylim[1] - ylim[0]), ylim[1] + epsilon_y * (ylim[1] - ylim[0]))

# Format axis scaling
x_scale = ax.get_xscale()
y_scale = ax.get_yscale()

if x_scale == 'log':
    ax.relim()
    ax.autoscale('x')
    ax.set_xscale('log')
if y_scale == 'log':
    ax.relim()
    ax.autoscale('y')
    ax.set_yscale('log')

# Control the line thickness for the axes
for spine in ax.spines.values():
    spine.set_linewidth(0.75)
# Now that we have taken all the information from the original figure and made a reformatted version in production quality
# we point the handle to the production quality figure and save it to the desired location
ax.minorticks_off()
xticks(ax, [0, 256, 512,768, 1024])
ax.set_yticks([0, 0.10, 0.20, 0.30], ['0.0', '0.1', '0.2', '0.3'])
ax.set_ylim(0 - (0.30*0.05) , 0.30 * 1.05)
plt.gca().xaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
ax.legend(['Post-batching', 'Pre-batching'], fontsize=4)
#plt.axvline(np.mean(data_2d_batching), color='k', linestyle='dashed', linewidth=0.2)
#plt.text(np.mean(data_2d_batching)*1.1, 220*0.9, '{}'.format(round(np.mean(data_2d_batching))), fontsize=3)
fig.savefig("edi-histo.pdf", format='pdf')
fig.show()


In [ ]:
output_2d_batching = get_operations("REDREC-CPU-V3-2D", 1, 54, 32, 1000, 1, 1, 1)
data_2d_batching = [int(x) for x in filter((lambda x: x != ' '), output_2d_batching.split("\n")[2].split(","))]
output_2d_non_batching = get_operations("REDREC-CPU-V3-2D", 1, 54, 32, 1000, 1, 0, 1)
data_2d_non_batching = [int(x) for x in filter((lambda x: x != ' '), output_2d_non_batching.split("\n")[2].split(","))]

In [ ]:

# Generate random data
data = np.random.randn(1000)

# Compute the histogram data
hist, bin_edges = np.histogram(data_2d_batching, bins=12, range= np.mean(data_2d_batching) + 6 * np.std(data_2d_batching) * np.array([-1, 1]))
hist_non, bin_edges_non = np.histogram(data_2d_non_batching, bins=12, range= np.mean(data_2d_non_batching) + 6 * np.std(data_2d_non_batching) * np.array([-1, 1]))


# Create a new fig and ax object to be a production quality version
fig, ax = plt.subplots()
# Set the font type and size
plt.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
fontsize = 7
plt.rc('pdf', fonttype=42)  # make the test exportable as text, not an image

# Calculate figure size in inches and set the size of our new figure
figure_width_cm = 4.3
figure_height_cm = 2.8
figure_width_in = figure_width_cm / 2.54
figure_height_in = figure_height_cm / 2.54

fig.set_size_inches((figure_width_in, figure_height_in))

# Set the margins of our production quality figure
left_margin = (4.3-3.48-0.0235)/4.3
right_margin = (4.3-0.05)/4.3
top_margin = (2.75-0.05)/2.8
bottom_margin = (2.75-2.21-0.0235)/2.8

ax.set_position([left_margin, bottom_margin, right_margin - left_margin, top_margin - bottom_margin])

ax.bar(bin_edges[:-1], hist, width=np.diff(bin_edges),  align='center', facecolor='#D9E3FD', edgecolor='#648EF7', linewidth=0.2)
ax.bar(bin_edges_non[:-1], hist_non, width=np.diff(bin_edges_non),  align='center', facecolor='#FFF1AD', edgecolor='#E0BB00', linewidth=0.2)
ax.set_xlabel('EDI cycles')
ax.set_ylabel('Frequency')
# Set a blank title for the new figure
ax.set_title('')

if True:
    ax.legend()
    legend = ax.legend(bbox_to_anchor=(0.95, 0.95), loc = 'upper right', borderaxespad=0., fontsize=4, handlelength=0.3, handleheight=0.3)

# Format the ticks as desired for the new figure
ax.tick_params(axis='both', direction='in', length = 2, labelsize = fontsize, pad = 2, width = 0.75)
ax.xaxis.set_ticks_position('both')
ax.yaxis.set_ticks_position('both')

xlim = ax.get_xlim()
ylim = ax.get_ylim()

ticks_x = np.linspace(xlim[0], xlim[1], 5)
ticks_y = np.linspace(ylim[0], ylim[1], 5)
ax.set_xticks(ticks_x)
ax.set_yticks(ticks_y)

ticklabels_x = ticks_x
ticklabels_y = ticks_y
ax.set_xticklabels(ticklabels_x)
ax.set_yticklabels(ticklabels_y)

tick_pad_x = 1.5
tick_pad_y = 1
ax.tick_params('x', labelsize=fontsize, pad=tick_pad_x) 
ax.tick_params('y', labelsize=fontsize, pad=tick_pad_y) 


# Formats the axes lables
label_x = ax.get_xlabel()
label_y = ax.get_ylabel()
label_pad_x = 1
label_pad_y = 1
ax.set_xlabel(label_x, fontsize=fontsize, labelpad=label_pad_x)
ax.set_ylabel(label_y, fontsize=fontsize, labelpad=label_pad_y)

# Sets the xlim and ylim axes so the the ticks are not right to the end (creating the corner square))
epsilon_x = 0.05
epsilon_y = epsilon_x
ax.set_xlim(xlim[0] - epsilon_x * (xlim[1] - xlim[0]), xlim[1] + epsilon_x * (xlim[1] - xlim[0]))
ax.set_ylim(ylim[0] - epsilon_y * (ylim[1] - ylim[0]), ylim[1] + epsilon_y * (ylim[1] - ylim[0]))

# Format axis scaling
x_scale = ax.get_xscale()
y_scale = ax.get_yscale()

if x_scale == 'log':
    ax.relim()
    ax.autoscale('x')
    ax.set_xscale('log')
if y_scale == 'log':
    ax.relim()
    ax.autoscale('y')
    ax.set_yscale('log')

# Control the line thickness for the axes
for spine in ax.spines.values():
    spine.set_linewidth(0.75)

# Now that we have taken all the information from the original figure and made a reformatted version in production quality
# we point the handle to the production quality figure and save it to the desired location
ax.minorticks_off()
xticks(ax, [0, 10, 20,  30, 40])
ax.set_yticks([0, 90, 180, 270, 360])
ax.set_ylim(0, 360 * 1.05)
plt.gca().xaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
ax.legend(['Post-batching', 'Pre-batching'], fontsize=4)
fig.savefig("edi-histo-1d.pdf", format='pdf')
fig.show()

In [ ]:
trials = 1000
reps = 1
atoms_dim = [(2,2), (4,4), (8,8), (12,12), (16,16), (24,24), (32,32)]
traps_dims = [(2,4), (4,8), (8,14), (12, 20), (16, 26), (24, 40), (32, 54)]

data_batched  = []
for i in range(len(atoms_dim)):
    print(f"{i}/{len(atoms_dim)}")
    write_time_list = []
    write_time_str = get_operations("REDREC-CPU-V3-2D", traps_dims[i][0], traps_dims[i][1], atoms_dim[i][0] * atoms_dim[i][1], trials, reps, 1, 0)
    data_batched += [(float(write_time_str.split('\n')[2].split(',')[0]), float(write_time_str.split('\n')[2].split(',')[1]))]
# Saving the results
file_name = "edi-batched-varying-traps-" + datetime.datetime.now().strftime("%Y-%m-%d, %H-%M-%S.pickle")
with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", file_name), "wb") as file:
    pickle.dump(
        {
            "xvals": [x * y for (x, y) in atoms_dim],
            "experiment": file_name,
            "data": data_batched
        }, file
    )

data_non_batched = []
for i in range(len(atoms_dim)):
    print(f"{i}/{len(atoms_dim)}")
    write_time_list = []
    write_time_str = get_operations("REDREC-CPU-V3-2D", traps_dims[i][0], traps_dims[i][1], atoms_dim[i][0] * atoms_dim[i][1], trials, reps, 0, 0)
    data_non_batched += [(float(write_time_str.split('\n')[2].split(',')[0]), float(write_time_str.split('\n')[2].split(',')[1]))]
# Saving the results
file_name = "edi-non-batched-varying-traps-" + datetime.datetime.now().strftime("%Y-%m-%d, %H-%M-%S.pickle")
with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", file_name), "wb") as file:
    pickle.dump(
        {
            "xvals": [x * y for (x, y) in atoms_dim],
            "experiment": file_name,
            "data": data_non_batched
        }, file
    )

In [ ]:
with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", "EDIs_batched.pickle"), "rb") as file:
    data_batched = pickle.load(file)
with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", "EDIs_non_batched.pickle"), "rb") as file:
    data_non_batched = pickle.load(file)

In [ ]:
from scipy.optimize import curve_fit

xvals = [2,4,8,12,16,24,32] 
yval_batched = [x[0] for x in data_batched]
yerr_batched = [x[1] for x in data_batched]
yval_non_batched = [x[0] for x in data_non_batched]
yerr_non_batched = [x[1] for x in data_non_batched]
fig, ax = plt.subplots() 
ax.set_xlabel("Width of the grid")
ax.set_ylabel("Number of EDI cycles")
model_batching, _ = curve_fit(func_linear, xvals, yval_batched)
model_non_batching, _ = curve_fit(func_quadratic, xvals, yval_non_batched)
PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [x * model_batching[1] for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#E06900", "lw": 1}
).draw()

PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [x**2 * model_non_batching[2]  for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#E0BB00", "lw": 1}
).draw()

prod_plot.production_quality_scatter(fig, ax, [xvals, xvals], [yval_batched, yval_non_batched], yerr_list= [yerr_batched, yerr_non_batched], markers=mark(["rrcbatch", "rrccpu"]), palette=pal(["rrcbatch", "rrccpu"]),   save_path=str(project_dir) + "/edi.pdf")

ax.legend(["Post-batching", "Pre-batching"],fontsize=4)
yticks(ax, [0,256,512,768,1024])
xticks(ax,[0,8,16,24,32])
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
plt.savefig("edi.pdf", format='pdf')
plt.show()
print(model_batching)
print(model_non_batching)

xvals = [2,4,8,12,16,24,32] 
yval_batched = [x[0]/y[0] for y,x in zip(data_batched, data_non_batched)]
fig, ax = plt.subplots() 
ax.set_xlabel("Width of the grid")
ax.set_ylabel("Relative cycles")
def func(x, a, b):
  return b * x + 1 
model_batching, _ = curve_fit(func, xvals, yval_batched)
PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [x * model_batching[1] + 1 for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#E0BB00", "lw": 1}
).draw()
prod_plot.production_quality_scatter(fig, ax, [[10222], xvals], [[2222222], yval_batched], markers=mark(["rrccpu", "rrccpu"]),  save_path=str(project_dir) + "/edi-relative.pdf")
print(pal(["rrcbatch", "rrccpu"]))
print(model_batching)

ax.get_legend().remove()
yticks(ax, [0,2,4,6, 8])
xticks(ax, [0,8,16,24,32])
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
plt.savefig("edi-relative.pdf", format='pdf')
plt.show()

In [ ]:
trials = 1000
reps = 1
atoms_dim = [(2,2), (4,4), (8,8), (12,12), (16,16), (24,24), (32,32)]
traps_dims = [(2,4), (4,8), (8,14), (12, 20), (16, 26), (24, 40), (32, 54)]

data_batched  = []
for i in range(len(atoms_dim)):
    print(f"{i}/{len(atoms_dim)}")
    write_time_list = []
    write_time_str = get_operations("REDREC-CPU-V3-2D", traps_dims[i][0], traps_dims[i][1], atoms_dim[i][0] * atoms_dim[i][1], trials, reps, 1, 0)
    data_batched += [(float(write_time_str.split('\n')[2].split(',')[0]), float(write_time_str.split('\n')[2].split(',')[1]), float(write_time_str.split('\n')[3].split(',')[0]), float(write_time_str.split('\n')[3].split(',')[1]))]
# Saving the results
file_name = "alpha-and-nu-batched-varying-traps-" + datetime.datetime.now().strftime("%Y-%m-%d, %H-%M-%S.pickle")
with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", file_name), "wb") as file:
    pickle.dump(
        {
            "xvals": [x * y for (x, y) in atoms_dim],
            "experiment": file_name,
            "data": data_batched
        }, file
    )

data_non_batched = []
for i in range(len(atoms_dim)):
    print(f"{i}/{len(atoms_dim)}")
    write_time_list = []
    write_time_str = get_operations("REDREC-CPU-V3-2D", traps_dims[i][0], traps_dims[i][1], atoms_dim[i][0] * atoms_dim[i][1], trials, reps, 0, 0)
    data_non_batched += [(float(write_time_str.split('\n')[2].split(',')[0]), float(write_time_str.split('\n')[2].split(',')[1]), float(write_time_str.split('\n')[3].split(',')[0]), float(write_time_str.split('\n')[3].split(',')[1]))]
# Saving the results
file_name = "alpha-and-nu-non-batched-varying-traps-" + datetime.datetime.now().strftime("%Y-%m-%d, %H-%M-%S.pickle")
with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", file_name), "wb") as file:
    pickle.dump(
        {
            "xvals": [x * y for (x, y) in atoms_dim],
            "experiment": file_name,
            "data": data_non_batched
        }, file
    )

In [ ]:
file_name = "displacement-non-batched" + datetime.datetime.now().strftime("%Y-%m-%d, %H-%M-%S.pickle")
with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", file_name), "wb") as file:
    pickle.dump(
        {
            "xvals": [x * y for (x, y) in atoms_dim],
            "experiment": file_name,
            "data": data_non_batched
        }, file
    )
    file_name = "displacement-batched" + datetime.datetime.now().strftime("%Y-%m-%d, %H-%M-%S.pickle")
with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", file_name), "wb") as file:
    pickle.dump(
        {
            "xvals": [x * y for (x, y) in atoms_dim],
            "experiment": file_name,
            "data": data_batched
        }, file
    )

In [ ]:
with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", "displacement-batched2024-07-17, 13-22-07.pickle"), "rb") as file:
    data_batched = pickle.load(file)["data"]
with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", "displacement-non-batched2024-07-17, 13-22-07.pickle"), "rb") as file:
    data_non_batched = pickle.load(file)["data"]

In [ ]:

xvals = [2,4,8,12,16,24,32] 
yval_batched = [x[2]/x[0] for x in data_batched]
yerr_batched = [x[1] for x in data_batched]
yval_non_batched = [x[2]/x[0] for x in data_non_batched]
yerr_non_batched = [x[1] for x in data_non_batched]
fig, ax = plt.subplots() 
ax.set_xlabel("Width of the grid")
ax.set_ylabel("Displacements per EDI cycles")
def func(x, a, b):
  return b * x + 1
def func_quad(x, a, b, c):
  return c * x ** 2 + 1
model_batching, _ = curve_fit(func_quad, xvals, yval_batched)
model_non_batching, _ = curve_fit(func, xvals, yval_non_batched)

PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [x**2 * model_batching[2] +1 for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#E06900", "lw": 1}
).draw()
PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [x * model_non_batching[1] + 1 for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#E0BB00", "lw": 1}
).draw()


prod_plot.production_quality_scatter(fig, ax, [xvals, xvals], [yval_batched, yval_non_batched],markers=mark(["rrcbatch", "rrccpu"]), palette=pal(["rrcbatch", "rrccpu"]),  save_path=str(project_dir) + "/displacement.pdf")

ax.legend(["Post-batching", "Pre-batching"],fontsize=4)
yticks(ax, [0, 25,50 ,75])
xticks(ax,[0,8,16,24,32])
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
plt.savefig("displacement.pdf", format='pdf')
plt.show()
print(model_batching)
print(model_non_batching)

xvals = [2,4,8,12,16,24,32] 
yval_batched = [x[0]/y[0] for y,x in zip(data_batched, data_non_batched)]
fig, ax = plt.subplots() 
ax.set_xlabel("Width of the grid")
ax.set_ylabel("Relative displacements per cycles")
def func(x, a, b):
  return b * x + 1 
model_batching, _ = curve_fit(func, xvals, yval_batched)
PlotPlotter(
    fig,
    ax,
    range(2,33,1),
    [x * model_batching[1] + 1 for x in range(2,33,1)],
    style = {"linestyle" : "-", "color": "#E06900", "lw": 1}
).draw()
prod_plot.production_quality_scatter(fig, ax, [[10222], xvals], [[2222222], yval_batched], markers=mark(["rrcbatch", "rrcbatch"]), palette=pal(["rrcbatch", "rrcbatch"]), save_path=str(project_dir) + "/displacement-relative.pdf")

print(model_batching)
ax.get_legend().remove()
yticks(ax, [0,2,4,6, 8])
xticks(ax, [0,8,16,24,32])
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
plt.savefig("displacement-relative.pdf", format='pdf')
plt.show()

In [ ]:
output_redrec = get_operations("REDREC-CPU-V3-2D", 32, 64, 1024, 100, 10, 1)
data_alpha_redrec_raw = [float(x) for x in filter((lambda x: x != ' '), output_redrec.split("\n")[0].split(","))]
data_nu_redrec_raw = [float(x) for x in filter((lambda x: x != ' '), output_redrec.split("\n")[1].split(","))]
data_alpha_redrec = [(float(x.split(",")[0]), float(x.split(",")[1])) for x in filter((lambda x: x != ' '), output_redrec.split("\n")[3].split("-"))]
data_nu_redrec = [(float(x.split(",")[0]), float(x.split(",")[1])) for x in filter((lambda x: x != ' '), output_redrec.split("\n")[4].split("-"))]
data_redrec = {"alpha_raw": data_alpha_redrec_raw, "nu_raw": data_nu_redrec_raw, "alpha": data_alpha_redrec, "nu": data_nu_redrec}
with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", "redrecv3-batching-loss-alpha-nu-ops.pickle"), "wb") as file:
    pickle.dump(data_redrec, file)

In [ ]:
output_redrec = get_operations("BIRD-CPU-2D", 32, 64, 1024, 100, 10, 1)
data_alpha_redrec_raw = [float(x) for x in filter((lambda x: x != ' '), output_redrec.split("\n")[0].split(","))]
data_nu_redrec_raw = [float(x) for x in filter((lambda x: x != ' '), output_redrec.split("\n")[1].split(","))]
data_alpha_redrec = [(float(x.split(",")[0]), float(x.split(",")[1])) for x in filter((lambda x: x != ' '), output_redrec.split("\n")[3].split("-"))]
data_nu_redrec = [(float(x.split(",")[0]), float(x.split(",")[1])) for x in filter((lambda x: x != ' '), output_redrec.split("\n")[4].split("-"))]
data_bird = {"alpha_raw": data_alpha_redrec_raw, "nu_raw": data_nu_redrec_raw, "alpha": data_alpha_redrec, "nu": data_nu_redrec}
with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", "bird-batching-loss-alpha-nu-ops.pickle"), "wb") as file:
    pickle.dump(data_bird, file)

In [ ]:
with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", "bird-alpha-nu-ops.pickle"), "rb") as file:
    data_bird = pickle.load(file) 
    data_alpha_bird = data_bird["alpha_raw"] 
    data_nu_bird = data_bird["nu_raw"] 

with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", "redrecv3-alpha-nu-ops.pickle"), "rb") as file:
    data_redrec = pickle.load(file) 
    data_alpha_redrec = data_redrec["alpha_raw"]
    data_nu_redrec = data_redrec["nu_raw"]
    
with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", "aro-alpha-nu-ops.pickle"), "rb") as file:
    data_aro = pickle.load(file) 
    data_alpha_aro = data_aro["alpha_raw"]
    data_nu_aro = data_aro["nu_raw"]

# Create a new fig and ax object to be a production quality version
fig, ax = plt.subplots()
plt.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
fontsize = 7
plt.rc('pdf', fonttype=42)  # make the test exportable as text, not an image
figure_width_cm = 4.3
figure_height_cm = 2.8
figure_width_in = figure_width_cm / 2.54
figure_height_in = figure_height_cm / 2.54
fig.set_size_inches((figure_width_in, figure_height_in))
left_margin = (4.3-3.48-0.0235)/4.3
right_margin = (4.3-0.05)/4.3
top_margin = (2.8-0.05)/2.8
bottom_margin = (2.8-2.21-0.0235)/2.8
ax.set_position([left_margin, bottom_margin, right_margin - left_margin, top_margin - bottom_margin])
ax.set_title('')

# modify existing Reds colormap with a linearly fading alpha
import matplotlib.pylab as pl
from matplotlib.colors import ListedColormap
red = pl.cm.RdPu  # original colormap
fading_red = red(np.arange(red.N)) # extract colors
fading_red[:, -1] = np.linspace(0, 1, red.N) # modify alpha
fading_red = ListedColormap(fading_red) # convert to colormap
blue = pl.cm.Blues  # original colormap
fading_blue = blue(np.arange(blue.N)) # extract colors
fading_blue[:, -1] = np.linspace(0, 1, blue.N) # modify alpha
fading_blue = ListedColormap(fading_blue) # convert to colormap
import matplotlib.colors as colors

def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap
orange = truncate_colormap(plt.get_cmap('YlOrBr'), 0.0, 0.5)  # original colormap
fading_orange = orange(np.arange(orange.N)) # extract colors
fading_orange[:, -1] = np.linspace(0, 1, orange.N) # modify alpha
fading_orange = ListedColormap(fading_orange) # convert to colormap


ax.hist2d(data_alpha_bird, data_nu_bird, bins=50, cmap=fading_blue, range=[[1800, 2200], [5000, 7000]])
ax.hist2d(data_alpha_redrec, data_nu_redrec, bins=50, cmap=fading_orange, range=[[1800, 2200], [5000, 7000]])
ax.hist2d(data_alpha_aro, data_nu_aro, bins=50, cmap=fading_red, range=[[1800, 2200], [5000, 7000]])
ax.set_xlabel('Transfers')
ax.set_ylabel('Displacements')

if True:
    ax.legend()
    legend = ax.legend(bbox_to_anchor=(0.95, 0.95), loc = 'upper right', borderaxespad=0., fontsize=4, handlelength=0.3, handleheight=0.3)
ax.tick_params(axis='both', direction='in', length = 2, labelsize = fontsize, pad = 2, width = 0.75)
ax.xaxis.set_ticks_position('both')
ax.yaxis.set_ticks_position('both')
xlim = ax.get_xlim()
ylim = ax.get_ylim()

ticks_x = np.linspace(xlim[0], xlim[1], 5)
ticks_y = np.linspace(ylim[0], ylim[1], 5)
ax.set_xticks(ticks_x)
ax.set_yticks(ticks_y)
ticklabels_x = ticks_x
ticklabels_y = ticks_y
ax.set_xticklabels(ticklabels_x)
ax.set_yticklabels(ticklabels_y)
tick_pad_x = 1.5
tick_pad_y = 1
ax.tick_params('x', labelsize=fontsize, pad=tick_pad_x) 
ax.tick_params('y', labelsize=fontsize, pad=tick_pad_y) 
label_x = ax.get_xlabel()
label_y = ax.get_ylabel()
label_pad_x = 1
label_pad_y = 1
ax.set_xlabel(label_x, fontsize=fontsize, labelpad=label_pad_x)
ax.set_ylabel(label_y, fontsize=fontsize, labelpad=label_pad_y)
epsilon_x = 0.05
epsilon_y = epsilon_x
ax.set_xlim(xlim[0] - epsilon_x * (xlim[1] - xlim[0]), xlim[1] + epsilon_x * (xlim[1] - xlim[0]))
ax.set_ylim(ylim[0] - epsilon_y * (ylim[1] - ylim[0]), ylim[1] + epsilon_y * (ylim[1] - ylim[0]))
x_scale = ax.get_xscale()
y_scale = ax.get_yscale()
for spine in ax.spines.values():
    spine.set_linewidth(0.75)
ax.minorticks_off()


ax.get_legend().remove()
plt.gca().xaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
fig.savefig("edi-histo.pdf", format='pdf')
fig.show()

In [ ]:
with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", "bird-batching-alpha-nu-ops.pickle"), "rb") as file:
    data_bird = pickle.load(file) 
    data_alpha_bird = data_bird["alpha_raw"] 
    data_nu_bird = data_bird["nu_raw"] 

with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", "redrecv3-batching-alpha-nu-ops.pickle"), "rb") as file:
    data_redrec = pickle.load(file) 
    data_alpha_redrec = data_redrec["alpha_raw"]
    data_nu_redrec = data_redrec["nu_raw"]
    
with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", "aro-batching-alpha-nu-ops.pickle"), "rb") as file:
    data_aro = pickle.load(file) 
    data_alpha_aro = data_aro["alpha_raw"]
    data_nu_aro = data_aro["nu_raw"]


fig, ax = plt.subplots()
plt.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
fontsize = 7
plt.rc('pdf', fonttype=42)  # make the test exportable as text, not an image
figure_width_cm = 4.3
figure_height_cm = 2.8
figure_width_in = figure_width_cm / 2.54
figure_height_in = figure_height_cm / 2.54
fig.set_size_inches((figure_width_in, figure_height_in))
left_margin = (4.3-3.48-0.0235)/4.3
right_margin = (4.3-0.05)/4.3
top_margin = (2.8-0.05)/2.8
bottom_margin = (2.8-2.21-0.0235)/2.8
ax.set_position([left_margin, bottom_margin, right_margin - left_margin, top_margin - bottom_margin])
ax.set_title('')

# modify existing Reds colormap with a linearly fading alpha
import matplotlib.pylab as pl
from matplotlib.colors import ListedColormap
red = pl.cm.RdPu  # original colormap
fading_red = red(np.arange(red.N)) # extract colors
fading_red[:, -1] = np.linspace(0, 1, red.N) # modify alpha
fading_red = ListedColormap(fading_red) # convert to colormap
blue = pl.cm.Reds  # original colormap
fading_blue = blue(np.arange(blue.N)) # extract colors
fading_blue[:, -1] = np.linspace(0, 1, blue.N) # modify alpha
fading_blue = ListedColormap(fading_blue) # convert to colormap
orange = pl.cm.YlOrBr  # original colormap
fading_orange = orange(np.arange(orange.N)) # extract colors
fading_orange[:, -1] = np.linspace(0, 1, orange.N) # modify alpha
fading_orange = ListedColormap(fading_orange) # convert to colormap


ax.hist2d(data_alpha_bird, data_nu_bird, bins=50, cmap=fading_blue, range=[[0,700], [500,1500]])
ax.hist2d(data_alpha_redrec, data_nu_redrec, bins=50, cmap=fading_orange, range=[[0,700], [500,1500]])
ax.hist2d(data_alpha_aro, data_nu_aro, bins=50, cmap=fading_red, range=[[0, 700], [500,1500]])
ax.set_ylabel('Displacements')
ax.set_xlabel('Transfers')

if True:
    ax.legend()
    legend = ax.legend(bbox_to_anchor=(0.95, 0.95), loc = 'upper right', borderaxespad=0., fontsize=4, handlelength=0.3, handleheight=0.3)
ax.tick_params(axis='both', direction='in', length = 2, labelsize = fontsize, pad = 2, width = 0.75)
ax.xaxis.set_ticks_position('both')
ax.yaxis.set_ticks_position('both')
xlim = ax.get_xlim()
ylim = ax.get_ylim()

ticks_x = np.linspace(xlim[0], xlim[1], 5)
ticks_y = np.linspace(ylim[0], ylim[1], 5)
ax.set_xticks(ticks_x)
ax.set_yticks(ticks_y)
ticklabels_x = ticks_x
ticklabels_y = ticks_y
ax.set_xticklabels(ticklabels_x)
ax.set_yticklabels(ticklabels_y)
tick_pad_x = 1.5
tick_pad_y = 1
ax.tick_params('x', labelsize=fontsize, pad=tick_pad_x) 
ax.tick_params('y', labelsize=fontsize, pad=tick_pad_y) 
label_x = ax.get_xlabel()
label_y = ax.get_ylabel()
label_pad_x = 1
label_pad_y = 1
ax.set_xlabel(label_x, fontsize=fontsize, labelpad=label_pad_x)
ax.set_ylabel(label_y, fontsize=fontsize, labelpad=label_pad_y)
epsilon_x = 0.05
epsilon_y = epsilon_x
ax.set_xlim(xlim[0] - epsilon_x * (xlim[1] - xlim[0]), xlim[1] + epsilon_x * (xlim[1] - xlim[0]))
ax.set_ylim(ylim[0] - epsilon_y * (ylim[1] - ylim[0]), ylim[1] + epsilon_y * (ylim[1] - ylim[0]))
x_scale = ax.get_xscale()
y_scale = ax.get_yscale()
for spine in ax.spines.values():
    spine.set_linewidth(0.75)
ax.minorticks_off()


ax.get_legend().remove()
plt.gca().xaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
fig.savefig("edi-histo-batching.pdf", format='pdf')
fig.show()

In [ ]:
with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", "bird-alpha-nu-ops.pickle"), "rb") as file:
    data_bird = pickle.load(file) 
    data_alpha_bird = data_bird["alpha"]
    data_nu_bird = data_bird["nu"] 

with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", "redrecv3-alpha-nu-ops.pickle"), "rb") as file:
    data_redrec = pickle.load(file) 
    data_alpha_redrec = data_redrec["alpha"]
    data_nu_redrec = data_redrec["nu"]

with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", "aro-alpha-nu-ops.pickle"), "rb") as file:
    data_redrec = pickle.load(file) 
    data_alpha_aro = data_redrec["alpha"]
    data_nu_aro = data_redrec["nu"]



xvals = [1,2,3,4,5,6,7,8,9] 
yvals_1 = [x[0] for x in data_alpha_redrec][:9]
print(yvals_1)
yerr_1 = [x[1] for x in data_alpha_redrec][:9]
yvals_2 = [x[0] for x in data_alpha_bird][:9]
yerr_2 = [x[1] for x in data_alpha_bird][:9]
yvals_3 = [x[0] for x in data_alpha_aro][:9]
yerr_3 = [x[1] for x in data_alpha_aro][:9]
fig, ax = plt.subplots()
prod_plot.production_quality_scatter(fig, ax, [xvals, xvals, xvals], [yvals_1, yvals_2, yvals_3], yerr_list=[yerr_1, yerr_2, yerr_3], markers=mark(["rrccpu", "bird", "aro"]), palette=pal(["rrccpu", "bird", "aro"]), save_path=str(project_dir) + "/loss.pdf")
ax.get_legend().remove()
xticks(ax, [1,2, 3,4, 5,6, 7,8, 9])
yticks(ax, [0,600,1200,1800, 2400])
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.0f'))
ax.legend(["Redrec operations", "Bird operations", "ARO Operations"], fontsize=4)
ax.set_xlabel("Cycle")
ax.set_ylabel("Transfers")
plt.savefig("transfers.pdf", format='pdf')
fig.show()

xvals = [1,2,3,4,5,6,7,8, 9] 
yvals_1 = [x[0] for x in data_nu_redrec][:9]
yerr_1 = [x[1] for x in data_nu_redrec][:9]
yvals_2 = [x[0] for x in data_nu_bird][:9]
yerr_2 = [x[1] for x in data_nu_bird][:9]
yvals_3 = [x[0] for x in data_nu_aro][:9]
yerr_3 = [x[1] for x in data_nu_aro][:9]
fig, ax = plt.subplots()
prod_plot.production_quality_scatter(fig, ax, [xvals, xvals, xvals], [yvals_1, yvals_2, yvals_3], yerr_list=[yerr_1, yerr_2, yerr_3], markers=mark(["rrccpu", "bird", "aro"]), palette=pal(["rrccpu", "bird", "aro"]), save_path=str(project_dir) + "/loss.pdf")
ax.legend(["Redrec operations", "Bird operations", "ARO Operations"], fontsize=4)
xticks(ax, [1,2, 3,4, 5,6, 7,8, 9])
yticks(ax, [0, 1500, 3000,4500, 6000])
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.0f'))
ax.set_xlabel("Cycle")
ax.set_ylabel("Displacements")
plt.savefig("displacements.pdf", format='pdf')
fig.show()
print(data_nu_redrec)
print(data_nu_bird)
print(data_nu_aro)
print(data_alpha_redrec)
print(data_alpha_bird)
print(data_alpha_aro)


In [ ]:
with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", "bird-batching-alpha-nu-ops.pickle"), "rb") as file:
    data_bird = pickle.load(file) 
    data_alpha_bird = data_bird["alpha"]
    data_nu_bird = data_bird["nu"] 

with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", "redrecv3-batching-alpha-nu-ops.pickle"), "rb") as file:
    data_redrec = pickle.load(file) 
    data_alpha_redrec = data_redrec["alpha"]
    data_nu_redrec = data_redrec["nu"]

with open(os.path.join(home, "LLRS", "resources", "operational-benchmarking", "aro-batching-alpha-nu-ops.pickle"), "rb") as file:
    data_redrec = pickle.load(file) 
    data_alpha_aro = data_redrec["alpha"]
    data_nu_aro = data_redrec["nu"]


xvals = [1,2,3,4,5,6,7,8, 9] 
yvals_1 = [x[0] for x in data_alpha_redrec][:9]
yerr_1 = [x[1] for x in data_alpha_redrec][:9]
yvals_2 = [x[0] for x in data_alpha_bird][:9]
yerr_2 = [x[1] for x in data_alpha_bird][:9]
yvals_3 = [x[0] for x in data_alpha_aro][:9]
yerr_3 = [x[1] for x in data_alpha_aro][:9]
fig, ax = plt.subplots()
prod_plot.production_quality_scatter(fig, ax, [xvals, xvals, xvals], [yvals_1, yvals_2, yvals_3], yerr_list=[yerr_1, yerr_2, yerr_3], markers=mark(["rrcbatch", "birdbatch", "aro"]), palette=pal(["rrcbatch", "birdbatch", "aro"]), save_path=str(project_dir) + "/loss.pdf")
ax.get_legend().remove()
xticks(ax, [1,2, 3,4, 5,6, 7,8, 9])
yticks(ax, [0, 200,400,600, 800])
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.0f'))
ax.legend(["Redrec batched moves", "Bird batched moves", "ARO Batched moves"], fontsize=4)
ax.set_xlabel("Cycle")
ax.set_ylabel("Transfers")
plt.savefig("transfers-batched.pdf", format='pdf')
fig.show()

xvals = [1,2,3,4,5,6,7,8, 9] 
yvals_1 = [x[0] for x in data_nu_redrec][:9]
yerr_1 = [x[1] for x in data_nu_redrec][:9]
yvals_2 = [x[0] for x in data_nu_bird][:9]
yerr_2 = [x[1] for x in data_nu_bird][:9]
yvals_3 = [x[0] for x in data_nu_aro][:9]
yerr_3 = [x[1] for x in data_nu_aro][:9]
fig, ax = plt.subplots()
prod_plot.production_quality_scatter(fig, ax, [xvals, xvals, xvals], [yvals_1, yvals_2, yvals_3], yerr_list=[yerr_1, yerr_2, yerr_3], markers=mark(["rrcbatch", "birdbatch", "aro"]), palette=pal(["rrcbatch", "birdbatch", "aro"]), save_path=str(project_dir) + "/loss.pdf")
ax.legend(["Redrec batched moves", "Bird batched moves", "ARO batched moves"], fontsize=4)
xticks(ax, [1,2, 3,4, 5,6, 7,8, 9])
yticks(ax, [0, 400,800,1200, 1600])
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.0f'))
ax.set_xlabel("Cycle")
ax.set_ylabel("Displacements")
plt.savefig("displacements-batched.pdf", format='pdf')
fig.show()
print(data_nu_redrec)
print(data_nu_bird)
print(data_nu_aro)
print(data_alpha_redrec)
print(data_alpha_bird)
print(data_alpha_aro)
